# SCC.413 Applied Data Mining
# Week 17
# Preprocessing

## Contents
- [Introduction](#intro)
- [Character Encoding](#encoding)
    - [Files](#files)
    - [Character variation](#variation)
- [Spelling Variation](#spelling)
- [Platform specific strings](#platform)
- [Text cleaning](#cleaning)
    - [FTFY: Fix That For You](#ftfy)
    - [Filtering with regular expressions](#regex)
- [Exercise](#exercise)

<a name="intro"></a>
## Introduction

Preprocessing is often understated in its importance for any data analysis task, especially when dealing with textual data. Ensuring that the data that you are analysing is consistent, has expected characteristics (i.e. what is actually present), and will be processed as expected by future stages of the analysis is essential.

It cannot be emphasised enough the value in actually examining samples of the data to see what is present. This can save a lot of work later on when you get nasty surprises when your analysis falls over, or you get unexpected results which you cannot explain (or, worse still, you're not aware of the noise propagating through your system).

This lab will discuss and demonstrate a few common issues that should be dealt with via pre-processing, and provide some solutions.

<a name="encoding"></a>
## Character Encoding

There are countless ways that text can be turned into binary data to be stored on disk, these are known as character encodings. Fortunately for us, more and more textual data is being standardised to UTF-8 (dominant Unicode character encoding), and the vast majority of web pages are now in UTF-8. An extra win is that Python 3 does everything in Unicode by default, and reads and writes to UTF-8 by default. This means a lot of the time you will not need to worry about character encoding, switching between them, and potentially corrupting text.

If the below code does not return 'UTF-8', then you will need to check your Python setup. In Windows, you might have 'cp1252' returned. Later versions of Python [let you fix this](https://dev.to/methane/python-use-utf-8-mode-on-windows-212i), or you may just need to declare 'utf-8' when reading/writing files. Better still, use Google Colab, or a non-Windows machine.

In [ ]:
import locale
locale.getpreferredencoding()

Because ASCII is a subset of Unicode (first block), if you are only dealing with ASCII characters, then you don't even need to know that you are using Unicode. However, if you want to use characters outside of ASCII, then Python makes this easy as everything is in Unicode. All 144,697 characters (as of [Unicode 14.0.0](https://www.unicode.org/versions/Unicode14.0.0/)) are available for you to use, so if you want a string with Jalapeño in it, you can go right ahead:

In [ ]:
print("This is a spicy jalapeño.")

Because Python 3 is Unicode throughout, you can even use non-ASCII characters as variable names ([with some restrictions](https://docs.python.org/3/reference/lexical_analysis.html#identifiers)):

In [ ]:
π = 3.14
print(π)

You can use the characters directly (as above) if you can get them on your keyboard, or pasted in from somewhere (e.g. [Unicode charts](https://unicode.org/charts/)). Alternatively, you can use special escape sequences:

- \xhh : char with 8-bit hex value (code points U+00 - U+FF (latin-1))
- \uhhhh : char with 16-bit hex value (code points U+0100 - U+FFFF)
- \Uhhhhhhhh : char with 32-bit hex value  (code points > U+10000)
- \N{name} : char with name from Unicode database: https://unicode.org/charts/

In [ ]:
a = "\xf1"
b = "\u03C0"
c = "\U0001F643"
d = "\N{SAMARITAN LETTER YUT}"

print(a,b,c,d)

Whichever you use, these equate to the same Unicode codepoint within Python:

In [ ]:
print("ñ" == "\xf1" == "\u00f1" == "\U000000f1" == "\N{LATIN SMALL LETTER N WITH TILDE}")

For I/O (e.g. reading/writing files), Python 3 uses UTF-8 by default. UTF-8, as described in the lecture, is a multi-byte character encoding capable of representing all possible Unicode characters. If we use Python's encode function to transform a string to a sequence of bytes, we can see this: 

In [ ]:
r = "jalapeño"
print(r.encode("utf-8"))

Defining the encoding as "utf-8" is actually redundant as it is the default (if not on Windows).

In [ ]:
print(r.encode())

The b'...' signifies the sequence of bytes.

In UTF-8, single byte characters are shown using their ASCII character (as for j, a, l, a, p, e, o). Multiple-byte characters are shown as hex values, so ñ is transformed to \xc3\xb1 (a two byte UTF-8 character). See the lecture for more detail on this.

We can encode (and output) to different character encodings, as long as it is possible to do so:

In [ ]:
r.encode("ascii")

1. Check you understand why the above code produces an error?
2. Try encoding to "latin-1", and check you understand why that is valid?
3. Try encoding the string "jalapeno" (without the diacritic) as ASCII. Does this work?

Fill in your answers here.
1. 
2. 
3. 

<a name="files"></a>
### Files

If you write to a file, again this will be done as UTF-8 by default (if not Windows). Try the below, and then download the file and open in a text editor or IDE which detects encoding (e.g. Geany, Atom). You will see that the unicode displays correctly when opened as UTF-8.

In [ ]:
with open ("test-file.txt", 'w') as f:
    f.write(r)

When you open files, again this will be done as UTF-8 by default:

In [ ]:
with open("test-file.txt") as f:
    print(f.read())

You can explicitly set the encoding for both reading and writing files, as below. This is generally considered good practice in case your code is ran on a Windows machine, or a system with the default encoding set to something other than UTF-8.

In [ ]:
with open("test-file.txt", encoding="utf-8") as f:
    print(f.read())

Try changing the encoding to "ascii" and "latin-1". Check you understand what happens, and write it down below.

In [ ]:
with open("test-file.txt", encoding="ascii") as f:
    print(f.read())

In [ ]:
with open("test-file.txt", encoding="latin-1") as f:
    print(f.read())

Answers:

1. ASCII 
2. LATIN-1 

Hopefully all of the text you deal with will be UTF-8, as this makes life easier with Python. If you do have text other than UTF-8, you will need to carefully check you know what the encoding is and read it as such. Switching between encodings can cause problems, and should be done with care. There's a good guide here: http://python-notes.curiousefficiency.org/en/latest/python3/text_file_processing.html

Note: Even when using UTF-8 it is generally good practice to explicitly set the encoding in case somebody using your code has changed the default encoding to something other than UTF-8.

There is a good overview available if you want to read more on Unicode and how it is dealt with in Python 3: https://docs.python.org/3/howto/unicode.html.

<a name="variation"></a>
### Character variation

One issue to be aware of is that there are often multiple Unicode codepoints available to represent the same character, and sometimes it is impossible to visually distinguish between them. For example, [combining diacritical marks](https://www.unicode.org/charts/PDF/U0300.pdf) can be used after any character to add a diacritic to the preceding letter. So, `e` with an acute accent can be written as `é` (one codepoint) or `e` and ` ́` combined (2 codepoints). When different codepoints are used, strings will not be equal, and if a different number of codepoints are used, their lengths will not be equal. i.e.:

In [ ]:
r = "jalapeño"
s = "jalape\xf1o"
t = "jalapen\u0303o" #combining n with  ̃: https://www.unicode.org/charts/PDF/U0300.pdf)

print(r, s, r == s)
print(s, t, s == t)
print(s,len(s), t, len(t))

Another example of this is quote marks. Many applications automatically "smart quote", replacing standard quotation marks with a fancier looking glyph. This can cause problems with tokenisation (for example) as the tokeniser is expecting one particular character to mean a quotation mark, and will miss the other varieties. Apostrophes can cause similar issues, and are also sometimes mixed up with quotation marks. The wide range of codepoints that can be used for quotemarks and apostrophes can be seen here: https://en.wikipedia.org/wiki/Quotation_mark#Unicode_code_point_table

<a name="spelling"></a>
## Spelling variation

With user-generated content, spelling variation can be a big issue, especially with the more informal language found on social networks and on internet forums. There are two main problems:

1. It is difficult to assign part-of-speech tags and other annotations automatically as many of these techniques rely heavily on a lexicon lookup, which often fails when words are spelt inconsistently or incorrectly.

2. When counting the occurrences of a particular word (or concept) the frequency is spread across the different variant spellings. If frequencies are incorrect it limits the reliability of statistics built upon these frequencies.

We are not going to directly cover dealing with spelling variation, but if you wish, you may want to look at [PyEnchant](https://github.com/rfk/pyenchant) and [PyHunSpell](https://github.com/blatinier/pyhunspell) for Python spellchecking libraries, and [VARD](http://ucrel.lancs.ac.uk/vard/about/), a Java tool originally developed for dealing with historical spelling variation, but can be used for dealing with modern spelling variation. 

<a name="platform"></a>
## Platform specific strings

A common issue is that textual data from the web contains things that are not "standard written language"; for example, hashtags, usernames, URLs, email addresses, emoticons, emojis, etc. Traditionally, NLP tools have been built to deal with clean edited language which has little variation. Many of these tools will not know what to do with some of the strings coming from social networks and other online sources. The situation is improving, with many NLP tools now having facilities for dealing with such things. Depending on the analysis to be undertaken, and the NLP pipeline being employed, it is sometimes desirable to remove these "troublesome" strings so they do not "break" existing tools, or the string may not be of interest for the study, or indeed they might cause problems for the study. For example, usernames being present could cause privacy issues, or bias author prediction based on language style. On the other hand, we may want to keep such features as they are useful to the task in hand, e.g. hashtags for topic analysis, emoji use for authorship analysis.

<a name="cleaning"></a>
## Text cleaning
There are many ways of dealing with the above issues, and decisions will be made based on the data being used, and the analysis being undertaken. Sometimes, it is enough (or all that is possible) to be aware of the issues and the impact they may have.

<a name="ftfy"></a>
### FTFY: Fix text for you
One particularly good tool worth having in your arsenal to deal with some of the above issues is [ftfy: fix text for you](https://ftfy.readthedocs.io/en/latest/). It is particularly good for dealing with corrupted UTF-8 text, and the problem of "mojibake", whereby characters are nonsensical because of mistakes made when encoding/decoding Unicode. The tool also normalises character usage, including smart quotes, and will also replace HTML entities which shouldn't be needed in UTF-8 text for NLP analysis.

For many cases, running ftfy's `fix_text` method will fix most of what you need, for more specific options [check the documentation](https://ftfy.readthedocs.io/en/latest/). To demonstrate, we'll output some text to a file (in UTF-8, as default), and read the file back in with the wrong encoding, "latin-1"

In [ ]:
!pip install ftfy

In [ ]:
import ftfy

In [ ]:
with open ("ftfy-test.txt", 'w', encoding="utf-8") as f:
    f.write("I’ve got a spicy jalapeño.")
    
with open ("ftfy-test.txt", encoding="latin-1") as f:
    garbled = f.read()
    
print(garbled)

Notice the 'smart' apostrophe is malformed due to it being a different codepoint within latin-1, as well as `ñ`.



In [ ]:
fixed = ftfy.fix_text(garbled)
print(fixed)

The encoding issues are fixed, back to standard Unicode. Also, the 'smart' apostrophe is replaced with a standard ascii apostrophe. Much nicer, and easier to deal with when it comes to tokenisation.

<a name="regex"></a>
### Filtering with regular expressions
Filtering unwanted strings can sometimes be dealt with during tokenisation (i.e. ignoring those sequences, such as emojis), but more often pre-processing will take place whereby the strings are removed or replaced. It is important to take care when removing strings from running text, that inadvertent damage is avoided, e.g. by replacing other wanted text unknowingly, or disrupting the flow of the text, which may impact annotation and other NLP tasks, as well as human readability. It is important to carefully decide what to replace strings with; a marker (e.g. `#hashtag`), a space, or nothing? Again, the data, and what analysis and tools are to be used should influence this decision.

Regular expressions are the obvious choice to replace such strings in most cases.

To demonstrate this, a single tweet is provided below, along with code to replace hashtags with just `#`.

In [ ]:
tweet = "This week we’re at a #careers event in #Blackpool @Pleasure_Beach, talking to students about #languages and language careers! Come have a go at some of our activities! 🌏#LoveLanguages #LoveLancaster @Lancaster_CI https://t.co/vQQWdrUuqh"
print(tweet)

In [ ]:
import re #regular expressions package

In [ ]:
#note we use r to denote a "raw string" for regular expression patterns, this is so we do not have to keep escaping \
hashtag = re.compile(r'#\w+') #compile the regular expression, good idea to do this in advance if used multiple times
cleaned = hashtag.sub("#", tweet) #use compiled regex to substitute matches for replacement "#", in tweet
print(cleaned)

You should be able to adapt the above code to replace usernames (@ mentions) with `@`. For both, try replacing with a space and removing completely. Can you process both in one regular expression?

In [ ]:
# answer



Review the resulting text each time, and judge the readability of the text. Does it change the text's meaning or nature? Make notes below.



**Advanced**: In one expression, can you replace `#hashtags` with `#` and `@mentions` with `@`, you'll need to use groupings and backreferences: https://docs.python.org/3/howto/regex.html

In [ ]:
# answer



<a name="exercise"></a>
## Exercise
Using text collected from last week's work, or the provided text from mumsnet (`mumsnet.txt`), examine the text and note any potential issues in the text that may cause problems for later analysis. Then write code to pre-process the text. Justify and describe decisions made below. Items you could consider are:

- user mentions
- hashtags
- urls
- emojis and emoticons
- smart quotes and apostrophes

In [ ]:
#exercise task code here, read in text, process, and output text to a new file.



Justify and describe decisions here:



